In [34]:
import os
import sys

import numpy as np
import matplotlib
%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib import colors
from hydra import initialize, compose
import pickle
from glob import glob
from pprint import pprint
from tqdm import tqdm
import speclite.filters
from astropy.io import ascii

# set paths to various directories based on the machine this code is being executed on
with initialize(version_base=None, config_path='config'):
    config = compose(config_name='config.yaml')  # overrides=['machine=uzay']

array_dir, data_dir, figure_dir, pickle_dir, repo_dir  = config.machine.array_dir, config.machine.data_dir, config.machine.figure_dir, config.machine.pickle_dir, config.machine.repo_dir

# enable use of local modules
if repo_dir not in sys.path:
    sys.path.append(repo_dir)

# set matplotlib style
plt.style.use(f'{repo_dir}/mejiro/mplstyle/science.mplstyle')

from mejiro.lenses.test import SampleStrongLens
from mejiro.plots import diagnostic_plot, plot, plot_util, overplot
from mejiro.analysis import stats
from mejiro.utils import util
from mejiro.helpers import gs, effarea
import mejiro

In [57]:
module_path = os.path.dirname(mejiro.__file__)
filter_responses_dir = os.path.join(module_path, 'data', 'filter_responses')
filter_response_files = sorted(glob(f'{filter_responses_dir}/*.ecsv'))

In [59]:
for f in filter_response_files:
    effarea.ecsv_to_csv(f)

In [36]:
filter_names = effarea._filter_name_list

In [37]:
filter = filter_names[0]

In [38]:
filter_response_files = sorted(glob(f'{filter_responses_dir}/RomanSCA*-{filter}.ecsv'))

data = ascii.read(filter_response_files[0])

In [39]:
total_data = [ascii.read(f) for f in filter_response_files]

In [40]:
wavelengths = []
responses = []

for file in filter_response_files:
    data = ascii.read(file)
    wavelengths.append(data['wavelength'])
    responses.append(data['response'])

In [41]:
total_min = np.min([np.min(w.value) for w in wavelengths])
total_max = np.max([np.max(w.value) for w in wavelengths])

print(f'Min wave: {total_min}, Max wave: {total_max}')

Min wave: 4710.0, Max wave: 7720.0


In [42]:
total_wavelengths = np.arange(total_min, total_max, 10)
total_responses = []

In [55]:
pad_wave = []
pad_response = []

for wavelength, response in zip(wavelengths, responses):
    min = np.min(wavelength).value
    max = np.max(wavelength).value
    # print(min, max)
    pad_before = int((min - total_min) / 10)
    pad_after = int((total_max - max) / 10)
    # print(pad_before, pad_after)
    
    if pad_before != 0:
        pad_before_vals = np.linspace(float(int(min) - pad_before), float(min)-10, pad_before - 1)
        print(pad_before_vals)
    if pad_after != 0:
        pad_after_vals = np.linspace(float(int(min) - pad_before), float(min)-10, pad_before - 1)
        print(pad_after_vals)

        padded_wavelength = np.pad(wavelength, pad_width=(pad_before, pad_after), mode=(pad_before_vals, pad_after_vals))
        padded_response = np.pad(response, (pad_before, pad_after), mode='constant', constant_values=(0, 0))

    # print(len(wavelength), len(padded))
    pad_wave.append(padded_wavelength)
    pad_response.append(padded_response)

[4728.]
[4728.]
[]
[]


TypeError: unhashable type: 'numpy.ndarray'

In [45]:
_ = [print(p[1], p[-1]) for p in pad_wave]

2365.0 Angstrom 7720.0 Angstrom
2365.0 Angstrom 7720.0 Angstrom
4720.0 Angstrom 0.0 Angstrom
4720.0 Angstrom 0.0 Angstrom
2365.0 Angstrom 0.0 Angstrom
4720.0 Angstrom 0.0 Angstrom
4720.0 Angstrom 0.0 Angstrom
4720.0 Angstrom 0.0 Angstrom
4720.0 Angstrom 0.0 Angstrom
2365.0 Angstrom 7720.0 Angstrom
2365.0 Angstrom 7720.0 Angstrom
4720.0 Angstrom 0.0 Angstrom
4720.0 Angstrom 7720.0 Angstrom
4720.0 Angstrom 0.0 Angstrom
4720.0 Angstrom 0.0 Angstrom
4720.0 Angstrom 0.0 Angstrom
4720.0 Angstrom 0.0 Angstrom
4720.0 Angstrom 0.0 Angstrom


In [46]:
data

wavelength,response
Angstrom,
float64,float64
4710.0,0.0
4720.0,4.420970641441537e-05
4730.0,0.0003094679449009076
4740.0,0.0014589203116757072
4750.0,0.005415689035765883
4760.0,0.016247067107297647
4770.0,0.039700316360145003
4780.0,0.07982062493122695


In [47]:
for filter in filter_names:
    filter_response_files = sorted(glob(f'{filter_responses_dir}/RomanSCA*-{filter}.ecsv'))